In [2]:
# import include nesscessary
import cv2
import pydicom

from joblib import Parallel, delayed
import glob
import time
import numpy as np
import os
from matplotlib import pyplot as plt

import torch
import random
import pandas as pd
import multiprocessing as mp
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from plotly.subplots import make_subplots
import time 
import streamlit as st

from PIL import Image, ImageOps

In [3]:
data_image = pd.read_csv('train.csv')

In [5]:
data_image[data_image['image_id'] == 1760744211]

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
4736,1,1511,1760744211,L,ML,63.0,0,0,0,0.0,0,C,49,True


In [6]:
data_image[data_image['image_id'] == 1273092690]

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
4735,1,1511,1273092690,L,CC,63.0,0,0,0,0.0,0,C,49,True


In [7]:
data_image[data_image['image_id'] == 1760744211]

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
4736,1,1511,1760744211,L,ML,63.0,0,0,0,0.0,0,C,49,True


### Cleaning Data 

In [4]:
data_image.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54706 entries, 0 to 54705
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   site_id                  54706 non-null  int64  
 1   patient_id               54706 non-null  int64  
 2   image_id                 54706 non-null  int64  
 3   laterality               54706 non-null  object 
 4   view                     54706 non-null  object 
 5   age                      54669 non-null  float64
 6   cancer                   54706 non-null  int64  
 7   biopsy                   54706 non-null  int64  
 8   invasive                 54706 non-null  int64  
 9   BIRADS                   26286 non-null  float64
 10  implant                  54706 non-null  int64  
 11  density                  29470 non-null  object 
 12  machine_id               54706 non-null  int64  
 13  difficult_negative_case  54706 non-null  bool   
dtypes: bool(1), float64(2)

In [5]:
def check_missing_data(df):
    list_miss = df.isnull().sum()
    # print(list_miss)
    list_miss = list_miss[list_miss >0 ]
    
    # Percentage of missing values

    precent_data = round( list_miss/(len(df)) * 100, 2)

    # Make table

    df_ = pd.DataFrame({'Quanlity Null': list_miss, 'Percentage': precent_data})
    
    return df_.sort_values(by='Percentage', ascending=False)

In [10]:
stats_null = check_missing_data(data_image)
stats_null

,Quanlity Null,Percentage
BIRADS,28420,51.95
density,25236,46.13
age,37,0.07


In [9]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=1)

fig.append_trace(go.Scatter(
    x=[3, 4, 5],
    y=[1000, 1100, 1200],
), row=1, col=1)

fig.append_trace(go.Scatter(
    x=[2, 3, 4],
    y=[100, 110, 120],
), row=2, col=1)

fig.append_trace(go.Scatter(
    x=[0, 1, 2],
    y=[10, 11, 12]
), row=3, col=1)


fig.update_layout(height=600, width=600, title_text="Stacked Subplots")
fig.show()

In [11]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "pie"}, {"type": "pie"}],
           [{"type": "pie"}, None]
           ],
    subplot_titles=("BIRADS chart percentage", "DESITY chart percentage", "AGE chart percentage"),
    horizontal_spacing=0.01,
    vertical_spacing= 0.2
)

fig.add_trace(go.Pie(values=[stats_null.Percentage[0], 100-stats_null.Percentage[0]], labels=['Null', 'Not null']),
              row=1, col=1)
fig.add_trace(go.Pie(values=[stats_null.Percentage[1], 100-stats_null.Percentage[1]], labels=['Null', 'Not null']),
              row=1, col=2)
fig.add_trace(go.Pie(values=[stats_null.Percentage[2], 100-stats_null.Percentage[2]], labels=['Null', 'Not null']),
              row=2, col=1)
fig.update_layout(height=500, width = 600)

In [12]:
## Phân tích đi tổng quan dữ liệu về density và BIRADS, cancer, difficult_negative_case 
fig = make_subplots(
    rows=3, cols=2, 
    shared_yaxes=True,
    specs=[[{}, {}],
           [{}, {}],
           [{"colspan": 2}, None]
    ],
    subplot_titles=("Statistical Density","Statistical BIRADS","Statistical cancer",'Statistical difficult negative case', 'Statistical views'),
    horizontal_spacing=0.1, 
    vertical_spacing= 0.2 
)

In [13]:
## 
def create_plot(lx, ly, px, py):
    fig.add_trace(go.Bar(x=lx, y=ly,
                    marker=dict(color=[4, 5, 6], coloraxis="coloraxis"), text=ly),
              px, py)
    

In [14]:
process_data_null = ["density" , "BIRADS", "cancer", "difficult_negative_case", 'view' ]
# process_data_null[0] = data_image.BIRADS.astype('str').value_counts()
# process_data_null[1] = data_image.density.astype('str').value_counts()
# process_data_null[2] = data_image.cancer.astype('str').value_counts()
id = 0
for i in range(1, 4):
    for j in range(1, 3):
        data = data_image[str(process_data_null[id])].astype('str').value_counts()
        create_plot(data.index, data.values, i, j)
        # print(data)
        id+=1
        if id == 5: break;
    if id == 5: break;

In [15]:
fig.update_layout(height=600,  showlegend = False,  title_text='Summary DataSet')
fig.show()

In [16]:
'''
- Tóm tắt thống kê cho chúng ta thấy dữ liệu null quá nhiều dẫn đến sự mất cân bằng về sữ liệu
- Đặc biệt khi chúng ta nhìn vào số lượng cancer quá ít chỉ 1158 bức ảnh
- Các trường hợp khó chuẩn đoán thì ít nhưng đạt ra câu hỏi bước ảnh như thế nào thì khó phát hiện ? có liên quan đến birabs và density hay không 

'''

'\n- Tóm tắt thống kê cho chúng ta thấy dữ liệu null quá nhiều dẫn đến sự mất cân bằng về sữ liệu\n- Đặc biệt khi chúng ta nhìn vào số lượng cancer quá ít chỉ 1158 bức ảnh\n- Các trường hợp khó chuẩn đoán thì ít nhưng đạt ra câu hỏi bước ảnh như thế nào thì khó phát hiện ? có liên quan đến birabs và density hay không \n\n'

In [17]:
## Phân tích mối quan hệ giữa cancer với birads
data_image.BIRADS = data_image.BIRADS.astype('str')
data_image.density = data_image.density.astype('str')
data_image.cancer = data_image.cancer.astype('str')
cancer_birads = data_image.groupby(['cancer'])['BIRADS'].value_counts()
cancer_birads_density = data_image.groupby(['cancer', 'BIRADS', 'density']).count()['site_id']
cancer_density = data_image.groupby(['cancer'])['density'].value_counts()
cancer_difficult_negative_case = data_image.groupby(['cancer'])['difficult_negative_case'].value_counts()
# cancer_difficult_negative_case

In [18]:
cancer_birads

cancer  BIRADS
0       nan       27926
        1.0       15772
        0.0        7585
        2.0        2265
1       0.0         664
        nan         494
Name: BIRADS, dtype: int64

In [19]:
cancer_density

cancer  density
0       nan        24742
        B          12342
        C          11898
        A           3052
        D           1514
1       nan          494
        B            309
        C            277
        A             53
        D             25
Name: density, dtype: int64

In [20]:
cancer_birads_density

cancer  BIRADS  density
0       0.0     A            380
                B           1956
                C           2406
                D            265
                nan         2578
        1.0     A           1909
                B           6914
                C           6066
                D            862
                nan           21
        2.0     A            257
                B           1026
                C            863
                D            108
                nan           11
        nan     A            506
                B           2446
                C           2563
                D            279
                nan        22132
1       0.0     A             53
                B            309
                C            277
                D             25
        nan     nan          494
Name: site_id, dtype: int64

In [21]:
cancer_difficult_negative_case

cancer  difficult_negative_case
0       False                      45843
        True                        7705
1       False                       1158
Name: difficult_negative_case, dtype: int64

In [22]:
cancer_birads = data_image.groupby(['cancer', 'view']).count()['image_id']

In [23]:
cancer_birads

cancer  view
0       AT         17
        CC      26199
        LM         10
        LMO         1
        ML          8
        MLO     27313
1       AT          2
        CC        566
        MLO       590
Name: image_id, dtype: int64

In [24]:
''''
- Các tấm hình cancer sẽ có birads loại 0(cần kiểm tra lại) và density tập trung ở B, C nhiều nhưng ko chiếm trọng số lớn trong bộ dữ liệu
- Các tấm hình cancer thì đề không bị khó chuẩn đoán
- Ngoài góc nhìn CC, MLO thì AT chiếm 2 tấm cancer
'''

"'\n- Các tấm hình cancer sẽ có birads loại 0(cần kiểm tra lại) và density tập trung ở B, C nhiều nhưng ko chiếm trọng số lớn trong bộ dữ liệu\n- Các tấm hình cancer thì đề không bị khó chuẩn đoán\n- Ngoài góc nhìn CC, MLO thì AT chiếm 2 tấm cancer\n"

In [25]:
### Display image 

def display_image(list_image, type_sick):
    # fig, axes = plt.subplot(int((len(list_image)+ 1)/2), 2, figsize=(20, 30))
    fig, axes = plt.subplots(int((len(list_image)+1)/2),2,figsize=(20, 30))
    axes = axes.flatten()
    i = 0
    for id in list_image:
        img = Image.open(f'train_images_main/{type_sick}/{data_image.patient_id[id]}/{data_image.laterality[id]}/{data_image.view[id]}/{data_image.image_id[id]}.png')
        axes[i].imshow(img, cmap='bone')
        axes[i].set_title(f'{data_image.view[id]}:{data_image.image_id[id]}')
        i+=1
    
    

In [26]:
# ls = data_image[(data_image['laterality'] == 'L') & (data_image['view'] == 'CC')&(data_image['cancer'] == '1')].index
# display_image(ls[:10], 'cancer')


In [27]:
data_image[(data_image['laterality'] == 'L') & (data_image['view'] == 'MLO')&(data_image['cancer'] == '1')][:10]

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
87,1,10130,388811999,L,MLO,71.0,1,1,1,0.0,0,B,49,False
90,1,10130,1672636630,L,MLO,71.0,1,1,1,0.0,0,B,49,False
190,1,10226,461614796,L,MLO,71.0,1,1,1,0.0,0,C,49,False
216,1,1025,1803952236,L,MLO,56.0,1,1,0,0.0,0,B,49,False
408,2,10432,1434858530,L,MLO,65.0,1,1,1,nan,0,nan,48,False
569,1,10589,195400299,L,MLO,74.0,1,1,1,0.0,0,B,170,False
570,1,10589,605115808,L,MLO,74.0,1,1,1,0.0,0,B,170,False
583,2,106,2018825992,L,MLO,65.0,1,1,1,nan,0,nan,21,False
607,2,10635,1095412840,L,MLO,68.0,1,1,1,nan,0,nan,29,False
610,2,10638,311168378,L,MLO,63.0,1,1,1,nan,0,nan,21,False
